In [3]:
import pandas as pd
import numpy as np
import requests
import re
import io


df = pd.read_csv('../../data/cleaned/cleaned_dataset.csv')

df['FireDiscoveryDateTime'] = pd.to_datetime(df['FireDiscoveryDateTime'], infer_datetime_format=True, errors = 'coerce')
df['ControlDateTime'] = pd.to_datetime(df['ControlDateTime'], infer_datetime_format=True, errors = 'coerce')

def build_request_param(id, lat, long, start_date, end_date, result_df, base_uri):
  print(id)
  params = f"latitude={lat}&longitude={long}&start={start_date.strftime('%Y%m%d')}&end={end_date.strftime('%Y%m%d')}"
  base_uri += params

  res = requests.get(base_uri)

  split_text = res.text.split('-END HEADER-')

  if len(split_text) >= 2:
    response_df = pd.read_csv(io.StringIO(split_text[1]))
    # response_df['DATE'] = response_df['YEAR'].apply(str) + '-' + response_df['MO'].apply(str) + '-' + response_df['DY'].apply(str)
    # response_df.drop(columns=['YEAR', 'MO', 'DY'], inplace=True)
    response_df['LAT'] = lat
    response_df['LONG'] = long
    response_df['ID'] = id
    result_df = pd.concat([result_df, response_df])

result_df = pd.DataFrame()
parameters='parameters=T2M,T2M_MAX,QV2M,PRECTOTCORR,WS2M,WS2M_MAX,WS10M,WS10M_MAX,GWETTOP,GWETPROF'
base_uri = f'https://power.larc.nasa.gov/api/temporal/daily/point?time-standard=lst&header=true&format=csv&community=sb&{parameters}&'
  

df.reset_index().apply(lambda row: build_request_param(row['index'], row['InitialLatitude'], row['InitialLongitude'], row['FireDiscoveryDateTime'], row['ControlDateTime'], result_df, base_uri), axis=1)

result_df.to_csv('../../data/cleaned/weather.csv')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
